In [1]:
import sys
import os

In [2]:
# Add the src folder to the path
sys.path.append(os.path.abspath(os.path.join("..", "src")))

In [3]:
# Now you can import the classes
from animals import Ant
from food import Meat

In [4]:
dog = Ant()
meat = Meat()

print(dog)
print(meat)

Ant (ATK: 2, HP: 5, TIER: 1)
Meat (Effect: Boost Attack)
